In [ ]:
import numpy as np  
import matplotlib.pyplot as plt  
from matplotlib.animation import FuncAnimation, FFMpegWriter  

# 定义常量  
q = 1.6e-25  # 电子电荷量，单位：库仑  
e_0 = 8.85e-18  # 真空介电常数，单位：F/m  
m_e = 9.11e-37  # 电子质量，单位：千克  
c = 300  # 光速，单位：米/秒  

# 定义电子的初始速度和加速度  
beta_init = 0.5  # 初始速度与光速的比值  
v_s = beta_init * c  # 初始速度  
a = 0.008  # 加速度，单位：米/秒^2  

# 定义电子的位置  
r_s = np.array([0, 0, 0], dtype=float)  # 电子位置，假设在原点  

# 定义观察点的网格  
x = np.linspace(-10, 10, 400)  
y = np.linspace(-10, 10, 400)  
X, Y = np.meshgrid(x, y)  
Z = np.zeros_like(X)  

# 初始化图形  
fig, ax = plt.subplots(figsize=(10, 8))  # 可以根据需要调整图形大小  
ax.set_xlim(-10, 10)  # 扩大x轴范围  
ax.set_ylim(-10, 10)  # 扩大y轴范围  
im = ax.imshow(np.zeros((400, 400)), cmap='viridis', extent=ax.get_xlim() + ax.get_ylim())  # 使用extent设置坐标轴范围  
ax.set_aspect('equal')  

# 定义动画更新函数  
def update(frame):  
    global r_s, v_s  # 声明全局变量以便更新  

    time_interval = 1 / 60  # 假设动画每秒60帧，每帧时间间隔  

    # 更新速度  
    v_s += a * time_interval  
    # 更新位置  
    r_s[0] += v_s * time_interval
    if r_s[0] > 10:  
        r_s[0] = r_s[0] % 10 - 10  
    
    # 重新计算观察点到电子位置的向量R  
    R = np.sqrt((X - r_s[0])**2 + (Y - r_s[1])**2 + (Z - r_s[2])**2)  
    R[R == 0] = 1  # 避免除以零  
    
    # 计算单位向量  
    n_s = np.stack((X - r_s[0], Y - r_s[1], Z - r_s[2]), axis=-1) / R[:, :, np.newaxis]  
    
    # 计算洛伦兹因子  
    beta_s = v_s / c  
    gamma = 1 / np.sqrt(1 - beta_s**2)  

    # 计算电场分量  
    E_x = (q / (4 * np.pi * e_0 * gamma**2)) * ( (n_s[:, :, 0] - beta_s)/ ((1 - beta_s * n_s[:, :, 0])**3) * R**2)
    E_y = (q / (4 * np.pi * e_0 * gamma**2)) * ( (n_s[:, :, 1] - beta_s)/ ((1 - beta_s * n_s[:, :, 1])**3) * R**2)
    E_z = (q / (4 * np.pi * e_0 * gamma**2)) * ( (n_s[:, :, 2] - beta_s)/ ((1 - beta_s * n_s[:, :, 2])**3) * R**2)
    
    # 计算电场强度的大小  
    E_magnitude = np.sqrt(E_x**2 + E_y**2 + E_z**2)  
    
    # 归一化电场强度  
    norm = plt.Normalize()  
    E_magnitude = norm(E_magnitude)  
 
    # 更新图像数据  
    im.set_data(E_magnitude.T)  # 确保数据是二维数组  
    
    return [im]  

# 创建FFMpegWriter实例  
writer = FFMpegWriter(fps=60, codec='libx264', extra_args=['-pix_fmt', 'yuv420p'])  

# 创建动画  
ani = FuncAnimation(fig, update, frames=np.linspace(0, 10, 10800), interval=200, blit=True)  

# 添加颜色量表  
cbar = plt.colorbar(im, ax=ax, orientation='vertical')  # 添加颜色量表  
cbar.set_label('Electric Field Magnitude')  # 设置颜色量表的标签  

# 保存动画  
ani.save('electron_field_animation.mp4', writer=writer)  
 
# 显示动画    
plt.show()

In [ ]:
import pandas as pd  
import numpy as np  
from scipy.ndimage import gaussian_filter1d  
import matplotlib.pyplot as plt  
import os  

# 读取Excel文件  
file_path = r"D:\Work\Experiment\密立根油滴.xlsx"  # 替换为你的Excel文件路径  
df = pd.read_excel(file_path) 

d = 0.005
ita = 0.0000183
s = 0.0016
rho_1 = 981
rho_2 = 1.2928
g = 9.794
b = 0.00823
p = 101325

# 初始化一个列表用于存储每列的平均值
column_means = []

# 遍历DataFrame的每一列
for column in df.columns:
    mean_value = df[column].mean()
    column_means.append(mean_value)
    
# 打印每列的平均值
print("每列的平均值为：")
for i, mean in enumerate(column_means):
    print(f"第{i+1}列的平均值: {mean}")
    
def calculate_q(u, t):
    r = (9 * ita * s / ((rho_1 - rho_2) * 2 * g * t))**(1/2)
    return 9 * 2**0.5 * np.pi * d * (((ita * s)**3 / ((rho_1 - rho_2) * g))**(1/2)) * (1/u) * (1 / t)**(3/2) * (1 + b / (p * r))**(-3/2)

# 初始化列表用于存储计算的电荷量
charges = []

# 读取电压U和时间t的平均值，并计算电荷量
for i in range(0, len(df.columns), 2):  # 假设电压U在奇数列，时间t在偶数列
    U_mean = df.iloc[:, i].mean()
    t_mean = df.iloc[:, i+1].mean()
    q = calculate_q(U_mean, t_mean)
    charges.append(q)

# 打印计算出的电荷量
print("计算出的电荷量为：")
for i, charge in enumerate(charges):
    print(f"第{i+1}对数据对应的电荷量: {charge}")
    
# 元电荷量
e_charge = 1.602e-19

# 初始化列表用于存储计算的电荷数
charge_numbers = []

# 使用之前计算的电荷量，计算电荷数
for charge in charges:
    charge_number = charge / e_charge
    charge_numbers.append(charge_number)

# 打印计算出的电荷数
print("计算出的电荷数为：")
for i, charge_number in enumerate(charge_numbers):
    print(f"第{i+1}对数据对应的电荷数: {charge_number}")